In [1]:
!pip install lxml
#!pip install beautifulsoup
#!wget https://dumps.wikimedia.org/enwiki/20190420/enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2
#!bunzip2 enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2    

In [2]:
# Count lines, words, characters in XML data file
!wc enwiki-20190420-pages-articles-multistream1.xml-p10p30302

 4586335 74563548 642819893 enwiki-20190420-pages-articles-multistream1.xml-p10p30302


In [3]:
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
def get_pages_from_xml_file(filename, start_tag='<page>', end_tag='</page>'):
    """Yields each page from the specified XML data file."""
    page = None
    with open(filename) as f:
        for line in f:
            if start_tag in line:
                page = []
                page.append(line)
            elif end_tag in line:
                page.append(line)
                page_xml = ''.join(page)
                yield page_xml
                page = None
            else:
                if page is not None:
                    page.append(line)

In [5]:
filename = 'enwiki-20190420-pages-articles-multistream1.xml-p10p30302'
pages = list(get_pages_from_xml_file(filename))

In [6]:
def get_title_from_page_xml(page_xml):
    """Returns the title of the given page."""
    soup = BeautifulSoup(page_xml, 'lxml')
    return soup.select_one('title').text

In [8]:
len(pages)  # Show the number of pages

19822

In [9]:
%time titles = [get_title_from_page_xml(page) for page in pages]

CPU times: user 45 s, sys: 940 ms, total: 46 s
Wall time: 46 s


In [10]:
titles[:5] + ['...'] + titles[-5:]  # Show the first and last 5 titles

['AccessibleComputing',
 'Anarchism',
 'AfghanistanHistory',
 'AfghanistanGeography',
 'AfghanistanPeople',
 '...',
 'The Lord of the Rings/One Ring',
 'Tax Freedom Day',
 'Tax',
 'Transhumanism',
 'TARDIS']

In [11]:
print(pages[0])  # Print the first page

  <page>
    <title>AccessibleComputing</title>
    <ns>0</ns>
    <id>10</id>
    <redirect title="Computer accessibility" />
    <revision>
      <id>854851586</id>
      <parentid>834079434</parentid>
      <timestamp>2018-08-14T06:47:24Z</timestamp>
      <contributor>
        <username>Godsy</username>
        <id>23257138</id>
      </contributor>
      <comment>remove from category for seeking instructions on rcats</comment>
      <model>wikitext</model>
      <format>text/x-wiki</format>
      <text xml:space="preserve">#REDIRECT [[Computer accessibility]]

{{R from move}}
{{R from CamelCase}}
{{R unprintworthy}}</text>
      <sha1>42l0cvblwtb4nnupxm6wo000d27t6kf</sha1>
    </revision>
  </page>



In [25]:
def get_text_from_page_xml(page):
    """Returns the text of the given page."""
    soup = BeautifulSoup(page, 'lxml')
    return soup.select_one('text')

In [26]:
page = pages[0]
#print(page)
soup = BeautifulSoup(page, 'lxml')
text = get_text_from_page_xml(page)
text, type(text)

(<text xml:space="preserve">#REDIRECT [[Computer accessibility]]
 
 {{R from move}}
 {{R from CamelCase}}
 {{R unprintworthy}}</text>, bs4.element.Tag)

In [27]:
data = []
for page in pages:
    row = {
        'title': get_title_from_page_xml(page),
        'text': get_text_from_page_xml(page),
    }
    data.append(row)

In [28]:
df = pd.DataFrame(data)
df.head()

,text,title
0,"<text xml:space=""preserve"">#REDIRECT [[Compute...",AccessibleComputing
1,"<text xml:space=""preserve"">{{redirect2|Anarchi...",Anarchism
2,"<text xml:space=""preserve"">#REDIRECT [[History...",AfghanistanHistory
3,"<text xml:space=""preserve"">#REDIRECT [[Geograp...",AfghanistanGeography
4,"<text xml:space=""preserve"">#REDIRECT [[Demogra...",AfghanistanPeople


### Challenge

#### 1. Create a Pandas dataframe containing the title and text of each page.

* Implement the `get_text_from_page_xml` function above.
* Re-create the dataframe with the text field filled in.

#### 2. Identify the five pages that have the _longest_ text.

* Find the length of each page's `<text>...</text>` element and add it to your dataframe.
* Sort the data frame by text length, descending.
* What are the titles of the five longest articles?

In [29]:
df_text = df

In [30]:
df_text['text_length'] = df['text'].astype(str).map(len)

In [32]:
df_text = df_text.sort_values('text_length',ascending=False)

In [33]:
df_text.head()

,text,title,text_length
10165,"<text xml:space=""preserve"">{{short description...",List of compositions by Johann Sebastian Bach,443556
15129,"<text xml:space=""preserve"">{{about|the country...",Pakistan,386911
15287,"<text xml:space=""preserve"">{{Redirect|Philippi...",Philippines,367556
3219,"<text xml:space=""preserve"">{{For2|the Republic...",China,325026
8838,"<text xml:space=""preserve"">{{short description...",History of India,324754
